
# REMBO: Random Embedding Bayesian Optimization

The REMBO (Random Embedding Bayesian Optimization) algorithm is a way to use Bayesian Optimization in extremely high-dimensional spaces. REMBO achieves this by projecting the high-dimensional optimization problem onto a randomly chosen low-dimensional subspace.

### REMBO Process Breakdown
1. **Random Matrix Generation**: We start by generating a random matrix, `A`, that maps from a low-dimensional space (`d` dimensions) to the original high-dimensional space (`D` dimensions). This matrix allows us to project points from the low-dimensional space to the original space.

2. **Projection to High-Dimensional Space**: Given a point in the low-dimensional space, we use the random matrix `A` to project it back to the high-dimensional space, where the actual optimization happens.

3. **Objective Function in Low Dimension**: REMBO defines an objective function in the low-dimensional space. It evaluates each low-dimensional point by projecting it to the high-dimensional space, then querying the high-dimensional objective function.

4. **Gaussian Process Modeling**: A Gaussian Process (GP) is used to approximate the unknown function in the low-dimensional space. GPs are beneficial because they provide an estimate of both the function value and uncertainty at each point.

5. **Optimization with Acquisition Function**: To select the best points to evaluate, REMBO uses an acquisition function in the low-dimensional space, which guides the optimization toward promising regions.

---
### Practice Problems

1. **Random Matrix Generation**: Writing a function to generate a random matrix for dimensionality reduction.
2. **Projection to High Dimension**: Implementing a function to project a low-dimensional point to the high-dimensional space.

Let's get started!


# Vanilla Bayeisan Optimization

In [1]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from scipy.optimize import minimize
from scipy.stats import norm


In [2]:
# Simple objective function (can be replaced with any black-box function)
def objective_function(x):
    return -np.sum(x**2)  # Simple quadratic function for demonstration

# Acquisition function (Expected Improvement)
def expected_improvement(x, model, best_f):
    x = x.reshape(1, -1)
    mu, sigma = model.predict(x, return_std=True)
    imp = mu - best_f
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # Return negative because we're minimizing



In [3]:
 def bayesian_optimization(n_iterations=10, d=5):
    # Initial random points
    X = np.random.uniform(-1, 1, size=(5, d))
    y = np.array([objective_function(x) for x in X])

    # GP model
    kernel = RBF(length_scale=1.0)
    model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5)

    for i in range(n_iterations):
        # Fit GP
        model.fit(X, y)
        best_f = np.max(y)

        # Optimize acquisition function
        x_next = minimize(
            lambda x: expected_improvement(x, model, best_f),
            x0=np.random.uniform(-1, 1, d),
            bounds=[(-1, 1)] * d
        ).x

        # Evaluate and update
        y_next = objective_function(x_next)
        X = np.vstack((X, x_next))
        y = np.append(y, y_next)

    return X, y

# Example usage
X, y = bayesian_optimization(n_iterations=100, d=5)
best_idx = np.argmax(y)
print(f"Best value: {y[best_idx]}")
print(f"Best point: {X[best_idx]}")

/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Best value: -0.02135295159269307
Best point: [-0.10338235 -0.02779744 -0.01198022  0.07143468  0.06816087]


# Generate Random Matrix

In [6]:
import numpy as np

def generate_random_matrix(d, d_low):
    """
    Generate a random matrix A of shape (d, d_low), where d is the original
    high dimension, and d_low is the low dimension.
    """
    A = np.random.rand(d, d_low)
    # TODO: Implement a line here to create a matrix with a standard normal distribution
    return A

In [7]:
# Example usage
d = 10  # Original dimension
d_low = 3  # Lower dimension

# Generate and analyze random matrix
A = generate_random_matrix(d, d_low)

# Demonstrate how to project a point from low to high dimension
y_low = np.random.uniform(-np.sqrt(d), np.sqrt(d), size=d_low)
y_high = A @ y_low

print(f"\nOriginal low-dim point: {y_low}")
print(f"Projected high-dim point: {y_high}")


Original low-dim point: [-1.28552181 -0.80885252  1.21068765]
Projected high-dim point: [-0.09200489 -0.2395214  -1.07459115 -0.0290603   0.00296578 -0.44396309
 -1.25552483 -0.84173392  0.56177476 -1.12673332]


# Implement REMBO

In [8]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from scipy.optimize import minimize

class REMBO:
    def __init__(self, d, d_low):
        self.d = d  # Original dimension
        self.d_low = d_low  # Lower dimension
        self.A = np.random.normal(0, 1, size=(d, d_low))

        # Set bounds for low-dimensional space
        self.bounds_low = [(-np.sqrt(d_low), np.sqrt(d_low))] * d_low

    def project_to_high_dim(self, y):
        """
        Project the low-dimensional point y to the high-dimensional space using matrix A.
        """
        # TODO: Implement a line to project y to the high-dimensional space by matrix multiplication
        high_dim_point = self.A @ y
        return high_dim_point

    def objective_low_dim(self, y, true_objective):
        """Objective function in low-dimensional space"""
        x = self.project_to_high_dim(y)
        return true_objective(x)

    def optimize(self, objective_function, n_iterations=10):
        # Initial points in low-dimensional space
        Y = np.random.uniform(-np.sqrt(self.d), np.sqrt(self.d),
                            size=(5, self.d_low))
        z = np.array([self.objective_low_dim(y, objective_function)
                     for y in Y])

        # GP model
        kernel = RBF(length_scale=1.0)
        model = GaussianProcessRegressor(kernel=kernel,
                                       n_restarts_optimizer=5)

        for i in range(n_iterations):
            # Fit GP
            model.fit(Y, z)
            best_f = np.max(z)

            # Optimize acquisition function
            y_next = minimize(
                lambda y: expected_improvement(y.reshape(1, -1),
                                            model, best_f),
                x0=np.random.uniform(-np.sqrt(self.d_low), np.sqrt(self.d_low),
                                   self.d_low),
                bounds=self.bounds_low
            ).x

            # Evaluate and update
            z_next = self.objective_low_dim(y_next, objective_function)
            Y = np.vstack((Y, y_next))
            z = np.append(z, z_next)

        return Y, z

# Example usage
def objective_function(x):
    return -np.sum(x**2)


In [9]:

# Get best result

X, y = bayesian_optimization(n_iterations=100, d=10)
best_idx = np.argmax(y)
print(f"Best value: {y[best_idx]}")
print(f"Best point: {X[best_idx]}")

# Initialize and run REMBO
rembo = REMBO(d=10, d_low=3)
Y, z = rembo.optimize(objective_function, n_iterations=100)

best_idx = np.argmax(z)
best_y = Y[best_idx]
best_x = rembo.project_to_high_dim(best_y)

print(f"Best value: {z[best_idx]}")
print(f"Best low-dim point: {best_y}")
print(f"Best high-dim point: {best_x}")

Best value: -0.2261163267641371
Best point: [ 0.20806467 -0.02543929 -0.09016376 -0.05441162 -0.12085477 -0.17849208
  0.04432647 -0.1634295   0.26594195 -0.15881965]
Best value: -0.10106748830992233
Best low-dim point: [0.05076884 0.04321723 0.09511027]
Best high-dim point: [-0.10596124 -0.07726102  0.18368555  0.11715059 -0.03351835  0.07677403
 -0.03404702 -0.04486394 -0.16136986  0.01326131]


# Solutions

In [ ]:
def generate_random_matrix(d, d_low):
    """
    Generate random matrix A ∈ R^(d×d_low) where d is original dimension
    and d_low is lower dimension
    """
    A = np.random.normal(0, 1, size=(d, d_low))
    return A

In [ ]:
def project_to_high_dim(self, y):
        """Project point from low to high dimension"""
        x = self.A @ y
        # Clip to original bounds [-1, 1]
        return np.clip(x, -1, 1)